In [1]:
#impot libraries
import pandas as pd
from requests import get
from time import sleep
from random import randint
from selenium import webdriver
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [2]:
#after cleaning of dataset
df1 = pd.read_csv('Amazon books data.csv')
df1.head()

,Unnamed: 0,CATEGORY,TITLE,AUTHOR,DATE,STAR,RATING
0,0,Arts & Photography,The 5 Love Languages: The Secret to Love that ...,Gary Chapman,"Jan 1, 2015",4.8 out of 5 stars,27397
1,1,Arts & Photography,The Life-Changing Magic of Tidying Up: The Jap...,by,"Oct 14, 2014",4.5 out of 5 stars,23167
2,2,Arts & Photography,Born a Crime: Stories from a South African Chi...,Trevor Noah,"Feb 12, 2019",4.8 out of 5 stars,20444
3,3,Arts & Photography,Born a Crime: Stories from a South African Chi...,Trevor Noah,"Nov 15, 2016",4.8 out of 5 stars,20444
4,4,Arts & Photography,Proof of Heaven: A Neurosurgeon's Journey into...,Eben Alexander,"Oct 23, 2012",4.3 out of 5 stars,13695


In [3]:
df1 = df1.drop('Unnamed: 0', axis = 1)

In [4]:
#change the column names to reflect the information about the data the hold
column_names = {'CATEGORY':'category', 'TITLE':'title', 'AUTHOR':'author','DATE':'publication_date', 'STAR':'rating','RATING':'number_of_ratings' }
df1= df1.rename(columns=column_names)
df1.head()

,category,title,author,publication_date,rating,number_of_ratings
0,Arts & Photography,The 5 Love Languages: The Secret to Love that ...,Gary Chapman,"Jan 1, 2015",4.8 out of 5 stars,27397
1,Arts & Photography,The Life-Changing Magic of Tidying Up: The Jap...,by,"Oct 14, 2014",4.5 out of 5 stars,23167
2,Arts & Photography,Born a Crime: Stories from a South African Chi...,Trevor Noah,"Feb 12, 2019",4.8 out of 5 stars,20444
3,Arts & Photography,Born a Crime: Stories from a South African Chi...,Trevor Noah,"Nov 15, 2016",4.8 out of 5 stars,20444
4,Arts & Photography,Proof of Heaven: A Neurosurgeon's Journey into...,Eben Alexander,"Oct 23, 2012",4.3 out of 5 stars,13695


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1536 entries, 0 to 1535
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   category           1536 non-null   object
 1   title              1536 non-null   object
 2   author             1536 non-null   object
 3   publication_date   1536 non-null   object
 4   rating             1536 non-null   object
 5   number_of_ratings  1536 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 72.1+ KB


In [6]:
df1.isnull().sum()

category             0
title                0
author               0
publication_date     0
rating               0
number_of_ratings    0
dtype: int64

**Categorical fields**

Categorical fields are fields that can have only a limited set of values. This is equivalent to "enum" types in most programming languages.In pandas they are expressed with the Categorical dtype.

If categorical dtypes are set on a DataFrame column, the corresponding field will have a constraints.enum property set wit the correct values when generating the table schema from the DataFrame object
The decision whether a field should be interpreted as categorical or not is sometimes not easy. Going further, we'll be inspecting the number of unique values present in the data is a good first criterion:

In [7]:
df1.apply(lambda s: s.nunique())

category               32
title                1001
author                699
publication_date      746
rating                  8
number_of_ratings    1039
dtype: int64

In this case, rating, and category all have below 50 values, so it makes sense to consider them as categorical. 

We can inspect the individual values for the categorical candidates to confirm:

In [8]:
def inspect_values(df, cols):
    separator = '-' * 50
    for col in cols:
        print(separator)
        display(df[col].value_counts())


df1.pipe(inspect_values, ['rating', 'category'])

--------------------------------------------------


4.7 out of 5 stars    421
4.8 out of 5 stars    327
4.6 out of 5 stars    300
4.5 out of 5 stars    157
4.4 out of 5 stars    114
4.9 out of 5 stars     90
-                      74
4.3 out of 5 stars     53
Name: rating, dtype: int64

--------------------------------------------------


Crafts, Hobbies & Home                        48
Test Preparation                              48
Science & Math                                48
Education & Teaching                          48
Travel                                        48
Cookbooks, Food & Wine                        48
Engineering & Transportation                  48
Religion & Spirituality                       48
Biographies & Memoirs                         48
Literature & Fiction                          48
Medical Books                                 48
Children's Books                              48
Christian Books & Bibles                      48
Sports & Outdoors                             48
Mystery, Thriller & Suspense                  48
Lesbian, Gay, Bisexual & Transgender Books    48
Arts & Photography                            48
Business & Money                              48
Reference                                     48
Parenting & Relationships                     48
Comics & Graphic Nov

In [9]:
df1['category'] = df1['category'].replace('Avg. Customer Review','Calendars')

#let us get these categoris again to confirm if the change has been effected
df1['category'].nunique(), list(df1['category'].unique())

(32,
 ['Arts & Photography',
  'Biographies & Memoirs',
  'Business & Money',
  'Calendars',
  "Children's Books",
  'Christian Books & Bibles',
  'Comics & Graphic Novels',
  'Computers & Technology',
  'Cookbooks, Food & Wine',
  'Crafts, Hobbies & Home',
  'Education & Teaching',
  'Engineering & Transportation',
  'Health, Fitness & Dieting',
  'History',
  'Humor & Entertainment',
  'Law',
  'Lesbian, Gay, Bisexual & Transgender Books',
  'Literature & Fiction',
  'Medical Books',
  'Mystery, Thriller & Suspense',
  'Parenting & Relationships',
  'Politics & Social Sciences',
  'Reference',
  'Religion & Spirituality',
  'Romance',
  'Science & Math',
  'Science Fiction & Fantasy',
  'Self-Help',
  'Sports & Outdoors',
  'Teen & Young Adult',
  'Test Preparation',
  'Travel'])

In [10]:
def set_as_categorical(df, cols):
    return df.astype({col: 'category' for col in cols})

df1 = df1.pipe(set_as_categorical, ['rating', 'category'])
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1536 entries, 0 to 1535
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   category           1536 non-null   category
 1   title              1536 non-null   object  
 2   author             1536 non-null   object  
 3   publication_date   1536 non-null   object  
 4   rating             1536 non-null   category
 5   number_of_ratings  1536 non-null   int64   
dtypes: category(2), int64(1), object(3)
memory usage: 53.0+ KB


Since some author names were not correctly appended during the scraping process.We'll correct that below;

In [11]:
df1[(df1['author']=='by ') |( df1['author']=='-')|(df1['author']=='---')]

,category,title,author,publication_date,rating,number_of_ratings
1,Arts & Photography,The Life-Changing Magic of Tidying Up: The Jap...,by,"Oct 14, 2014",4.5 out of 5 stars,23167
11,Arts & Photography,Strange Planet,by,"Nov 19, 2019",4.9 out of 5 stars,9879
36,Business & Money,The Life-Changing Magic of Tidying Up: The Jap...,by,"Oct 14, 2014",4.5 out of 5 stars,23167
103,Comics & Graphic Novels,Strange Planet,by,"Nov 19, 2019",4.9 out of 5 stars,9879
110,Comics & Graphic Novels,The Legend of Zelda: Hyrule Historia,by,"Jan 29, 2013",4.9 out of 5 stars,5533
121,Computers & Technology,The Legend of Zelda: Hyrule Historia,by,"Jan 29, 2013",4.9 out of 5 stars,5533
141,"Cookbooks, Food & Wine","The Plant Paradox: The Hidden Dangers in ""Heal...",by,"Jul 11, 2017",4.4 out of 5 stars,7451
145,"Crafts, Hobbies & Home",The Life-Changing Magic of Tidying Up: The Jap...,by,"Oct 14, 2014",4.5 out of 5 stars,23167
185,Engineering & Transportation,"Guns, Germs, and Steel: The Fates of Human Soc...",by,"Mar 7, 2017",4.5 out of 5 stars,4498
248,Law,The Guilty,by,"Jan 1, 1996",4.5 out of 5 stars,6178


In [12]:
#Hence, we manually correct author names with anomalies
df1.iloc[1,2] = df1.iloc[1,2].replace("by ", "Marie Kondo")
df1.iloc[11,2] = df1.iloc[11,2].replace("by ", "Nathan W. Pyle")
df1.iloc[36,2] = df1.iloc[36,2].replace("by ", "Marie Kondo")
df1.iloc[103,2] = df1.iloc[103,2].replace("by ", "Nathan W. Pyle")
df1.iloc[110,2] = df1.iloc[110,2].replace("by ", "Patrick Thorpe, Michael Gombos, et al")
df1.iloc[121,2] = df1.iloc[121,2].replace("by ", "Patrick Thorpe, Michael Gombos, et al")
df1.iloc[141,2] = df1.iloc[141,2].replace("by ", "Dr. Steven R Gundry MD")
df1.iloc[145,2] = df1.iloc[145,2].replace("by ", "Marie Kondo")
df1.iloc[175,2] = df1.iloc[175,2].replace("_", " Yuval Noah Harari and Janaína Marcoantonio")
df1.iloc[185,2] = df1.iloc[185,2].replace("by ", "Jared Diamond, Doug Ordunio, et al.")
df1.iloc[248,2] = df1.iloc[248,2].replace("by ", "David Baldacci")
df1.iloc[344,2] = df1.iloc[344,2].replace("by ", "Marie Kondo")
df1.iloc[373,2] = df1.iloc[373,2].replace("by ", "Marie Kondo")
df1.iloc[388,2] = df1.iloc[388,2].replace("by ", "E L James, Zachary Webber, et al.")
df1.iloc[437,2] = df1.iloc[437,2].replace("by ", "Marie Kondo")
df1.iloc[489,2] = df1.iloc[489,2].replace("_", "Yuval Noah Harari and Janaína Marcoantonio")
df1.iloc[527,2] = df1.iloc[527,2].replace("by ", "Patrick Thorpe, Michael Gombos, et al.")
df1.iloc[572,2] = df1.iloc[572,2].replace("---", "NO AUTHOR")
df1.iloc[580,2] = df1.iloc[580,2].replace("by ", " Bill Martin Jr. and Eric Carle")
df1.iloc[770,2] = df1.iloc[770,2].replace("by ", "John Green ")
df1.iloc[771,2] = df1.iloc[771,2].replace("by ", "John Green")
df1.iloc[872,2] = df1.iloc[871,2].replace("by ", "Bill Martin Jr. and Eric Carle")
df1.iloc[1004,2] = df1.iloc[1003,2].replace("by ", "Linda Anne Silvestri PhD RN FAAN and Angela Elizabeth Silvestri PhD APRN FNP-BC CNE")
df1.iloc[1060,2] = df1.iloc[1059,2].replace("---", "The Driver and Vehicle Standards Agency The Driver and Vehicle Standards Agency")
df1.iloc[1124,2] = df1.iloc[1123,2].replace("by ", "Junji Ito")
df1.iloc[1168,2] = df1.iloc[1167,2].replace("by ", "Louisa May Alcott ")
df1.iloc[1229,2] = df1.iloc[1228,2].replace("by ", "Rachel Hollis and HarperCollins Leadership")
df1.iloc[1319,2] = df1.iloc[1318,2].replace("by ", "Travis Bradberry, Jean Greaves, et al.")
df1.iloc[1435,2] = df1.iloc[1434,2].replace("by ", "Jared Diamond, Doug Ordunio, et al.")
df1.iloc[1448,2] = df1.iloc[1447,2].replace("by ", " Nathan W. Pyle")

In [13]:
#check
df1[(df1['author']=='by ') |( df1['author']=='-')|(df1['author']=='---')]

,category,title,author,publication_date,rating,number_of_ratings


**Publication Date**

In [14]:
from datetime import datetime
df1[df1['publication_date']=='-']

,category,title,author,publication_date,rating,number_of_ratings
211,History,To Kill a Mockingbird,Harper Lee,-,4.8 out of 5 stars,27800
339,Politics & Social Sciences,To Kill a Mockingbird,Harper Lee,-,4.8 out of 5 stars,27800
839,Parenting & Relationships,Knock Knock What I Love about You Fill in the ...,Knock Knock,-,4.7 out of 5 stars,6016
1060,Business & Money,Official Highway Code,Sheryl Sandberg,-,4.7 out of 5 stars,6069
1172,"Crafts, Hobbies & Home",Knock Knock What I Love about You Fill in the ...,Knock Knock,-,4.7 out of 5 stars,6014


In [15]:
#We replace them with the necessary values
df1.iloc[211,3] = df1.iloc[211,3].replace("-", "Jul 5, 2005")
df1.iloc[339,3] = df1.iloc[339,3].replace("-", "Jul 5, 2005")
df1.iloc[839,3] = df1.iloc[838,3].replace("-", "Jun 15, 2015")
df1.iloc[1060,3] = df1.iloc[1059,3].replace("-", "Jun 1, 2015")
df1.iloc[1172,3] = df1.iloc[1171,3].replace("-", "Jun 15, 2015")

In [16]:
#converting to standard datetime format, we get
df1['publication_date'] = pd.to_datetime(df1['publication_date'], format='%b %d, %Y', errors='coerce')

**Rating**

In [17]:
#We are intersted in the number part of the column. split the column and appennd the first element
df1['rating'] = df1['rating'].apply(lambda x: x.split(' ')[0])

#We replace all occurence of '-' in rating with null values and convert the column to float data type
df1['rating'] = df1['rating'].replace('-', np.nan).astype(float)

**Table schema**

Creating table schema from the DataFrame. The build_table_schema() function allows to get;

1. The output of build_table_schema() is a Python object (dictionary). It can be modified (either manually or programmatically) from within the notebook

2. It must be converted to a string (for copy-pasting) or saved to a file to be used with other applications. Use the json module of the standard library for this.

In [18]:
from pandas.io.json import build_table_schema
import json

def get_table_schema(df):
    return build_table_schema(df1, index=False, version=False)

json.dumps(df1.pipe(get_table_schema))

'{"fields": [{"name": "category", "type": "any", "constraints": {"enum": ["Arts & Photography", "Biographies & Memoirs", "Business & Money", "Calendars", "Children\'s Books", "Christian Books & Bibles", "Comics & Graphic Novels", "Computers & Technology", "Cookbooks, Food & Wine", "Crafts, Hobbies & Home", "Education & Teaching", "Engineering & Transportation", "Health, Fitness & Dieting", "History", "Humor & Entertainment", "Law", "Lesbian, Gay, Bisexual & Transgender Books", "Literature & Fiction", "Medical Books", "Mystery, Thriller & Suspense", "Parenting & Relationships", "Politics & Social Sciences", "Reference", "Religion & Spirituality", "Romance", "Science & Math", "Science Fiction & Fantasy", "Self-Help", "Sports & Outdoors", "Teen & Young Adult", "Test Preparation", "Travel"]}, "ordered": false}, {"name": "title", "type": "string"}, {"name": "author", "type": "string"}, {"name": "publication_date", "type": "datetime"}, {"name": "rating", "type": "number"}, {"name": "number_o

In [19]:
df1 = df1.to_csv('Newest_amazon_data.csv', index=False, encoding='utf-8')